In [1]:
import acquire as a
import prepare as p
import wrangle as w
import explore as e
import model as m
import evaluate as eval

import seaborn as sns
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures


from sklearn.preprocessing import MinMaxScaler, RobustScaler

https://pen.do/support/difference-between-property-and-unit/

# Acquire

In [2]:
#Acquire data from SQL using my function

In [3]:
sql_query = '''
SELECT  parcelid, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
taxvaluedollarcnt 
FROM properties_2017
JOIN predictions_2017 as pred USING (parcelid)
WHERE pred.transactiondate >= '2017-05-01' AND pred.transactiondate <= '2017-08-31'
AND (propertylandusetypeid > 259 AND propertylandusetypeid  < 265);
'''

In [ ]:
df= a.get_data_from_sql('zillow',sql_query)

In [ ]:
#check my df
df.head()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()

**takeaways**
- My sql query filter:
   - single unit property .
   - transaction during  May-August, 2017
- calculatedfinishedsquarefee and taxvaluedollarcnt have missing values

# Prepare

In [ ]:
#use a function that gives us a quick report
w.miss_dup_values(df)

In [ ]:
# I'm going to drop duplicates

In [ ]:
df = df.drop_duplicates()

In [ ]:
#I'm going to drop the rows with missing values

In [ ]:
df = df.dropna(how='any',axis=0)

In [ ]:
w.miss_dup_values(df)

In [ ]:
df.info()

In [ ]:
# I'm going to change parcelid to string, it is a uniqure identifier for parcels lots
df['parcelid'] = df['parcelid'].astype('str')

In [ ]:
df.dtypes

In [ ]:
df.rename(columns={'bedroomcnt': 'n_bedrooms', 
                    'bathroomcnt': 'n_bathrooms',
                    'calculatedfinishedsquarefeet':'sq_ft',
                     'taxvaluedollarcnt': 'assessed_value_usd'}, inplace=True)

In [ ]:
#checking the uniques values for each column
columns = df.columns.tolist()
print( 'Columns')
print(" ")
cat_list = []
for col in columns:
    print(f'{col} --> {df[col].nunique()} unique values')
    if df[col].nunique() < 26:
        cat_list.append(col)
    print(" ")


In [ ]:
#checking  the variables that have few values
for l in cat_list:
    print(l)
    print(df[l].value_counts().sort_index())
    print("--------------------------- ")
    print(" ")

** Takeaways**
- change parcelid to object type
- there are property whith 0 bedrooms and bathtooms. I will check this more

In [ ]:
#explore n_bedrooms == 0 & n_bathroos == 0

In [ ]:
df[(df.n_bedrooms == 0) & (df.n_bathrooms == 0)].count()

In [ ]:
df[(df.n_bedrooms == 0) & (df.n_bathrooms == 0)]

In [ ]:
df.shape

**I decided to keep the properties with 0 bathrooms and bedrooms it represents a small percentage**

In [ ]:
#checking calculatedfinishedsquarefeet
df.sq_ft.sort_values().head(20)

In [ ]:
df[(df.sq_ft <400)]

In [ ]:
df.info()

**takeaway**
- I decided to keep bathrooms and bedrooms = 0


In [ ]:
#pd.cut(df.n_bedrooms,  [0, 2, 4, 12])


In [ ]:
df.describe().T

In [ ]:
p.distribution(df)

because all my features are continous i decided to boxplot

In [ ]:
p.distribution_boxplot(df)

**Takeaways** 
-  bathrooms , bedrooms, sq_dt and usd_valure are continous.
- target is  usd_value


### split train, validate, test

In [ ]:
train, validate, test = p.split_data(df)

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = p.split_Xy(train,validate,test, 'assessed_value_usd')

In [ ]:
#Scale our data
X_train_scaled, X_validate_scaled, X_test_scaled = p.scaled_df(X_train, X_validate, X_test, RobustScaler())

**takeaways**
- I use Robust Scaler that uses parameters that are more robust to outliers,  because  bathrooms , bedrooms, and sq_dt have outliers.

# Expolore


before Exploration this is what I think:



- The variables that can influence the value of home are square feet, and number of bedrooms and at last number of bathrooms. 
- square feet, and number of bedrooms  can have an influence no matter where the property located.

-  other factors can influence the value of a home such as zip code, year built,  school district.

In [ ]:
#  I want to explore with my target

In [ ]:
plt.figure(figsize=(13, 6))
train['assessed_value_usd'].hist(grid=False,)


In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(train['assessed_value_usd'])

In [ ]:
plt.figure(figsize=(16, 8))
sns.displot( train['assessed_value_usd'])


In [ ]:
# I creating a new df 

In [ ]:
train_scaled = pd.concat([X_train_scaled, pd.DataFrame( {'assessed_valure_usd': y_train}) ], axis= 1 )

In [ ]:
train_scaled.head()

In [ ]:
e.heatmap(train_scaled)

Features in order with more correlation with the target:

    - 1. sq_ft
    - 2. n_bathrooms
    - 3. n_bedrooms

In [ ]:
sns.pairplot(train_scaled, kind="reg", plot_kws={'line_kws':{'color':'red'}}, corner=True)

**Takeaways**
- sq_ft has more correlation with our target
- sq_ft has higher correlation n_bathrooms

## Statistical Tests

### - T-test

 **Is any diference in  the average of assessed_value_usd  for 2   bedrooms  vs 3 o bedrooms properties?**


 - 𝐻𝑜 : There is no difference in  the average of assessed_value_usd  for the properties with  3  bedrooms  vs 2 bedrooms
- 𝐻𝑎 : There is significant  difference in  the average of assessed_value_usd  for the properties with  3  bedrooms  vs 2 bedrooms



- **continous** = assessed_value_usd 
- **categorical** = 2 groups ( bedroom_2, bedroom_3)  
- **continous vs categorical** = 2-tailed (significantly different) , 2-sample (comparing 2 groups) t-test.

1. Set Significance Level: $\alpha = .05$ (in other words Confidence level is 0.95)

2. Verify Assumptions:

    - Normal Distribution, or at least 30 observations and "kinda" normal. The more observations you have, the less "normal" it needs to appear. (CLT)
    - Independent samples
    - Equal Variances (or set method argument to False when not)
3. Compute test statistic and probability (t-statistic & p-value) using stats.ttest_ind

In [ ]:
# Set Significance Level
alpha = 0.05

In [ ]:
train_scaled.head()

In [ ]:
#create my groups
bedroom_2 = train_scaled[train_scaled.n_bedrooms == 2].assessed_valure_usd
bedroom_3 = train_scaled[train_scaled.n_bedrooms ==3 ].assessed_valure_usd

In [ ]:
#Normal Distribution
bedroom_2.shape, bedroom_3.shape

In [ ]:
# Equal Variances (or set method argument to False when not)
bedroom_2.var(), bedroom_3.var()

In [ ]:
#Compute Test Statistic
t, p = stats.ttest_ind(bedroom_2, bedroom_3, equal_var = False)
t,p

In [ ]:
if (p < alpha):
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")

t is negative that means the average in assessed_value_usd is for 3 bedrooms is  greater  than 2 bedrooms

In [ ]:
print('Average  assessed value  for 2 bedrooms is : $ ',round( bedroom_2.mean(), 2))
print('Average  assessed value  for 3 bedrooms is : $', round (bedroom_3.mean(), 2))

### - Correlation

Are bathrooms and bedrooms linearly correlated? 
- 𝐻𝑜 : There is not a linear correlation between number of bathrooms and number of bedrooms for a property.
- 𝐻𝑎 : There is a linear correlation between number of bathrooms and number of bedrooms for a property.


In [ ]:
r, p = stats.pearsonr(train_scaled.n_bathrooms, train_scaled.n_bedrooms)
r, p

In [ ]:
if (p < alpha):
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")

In [ ]:

sns.lmplot(x = 'n_bathrooms', y= 'n_bedrooms', data = train_scaled, line_kws={'color': 'red'})
plt.xlabel('Bathrooms')
plt.ylabel('Bedrooms')
plt.title('Bathrooms vs Bedrooms')
plt.annotate(f'Pearson r:{r:.4f}\n p-stat: {p:.2f}', xy =(-3, 8))

**Takeaways**
- What independent variables are correlated with assessed_value_usd?
    - my surprise is bathrooms have higher correlation with our target than bedrooms
    - **sq_ft  looks to be the best predictor of our target variable**

- Which independent variables are correlated with other independent variables?
    - bathrooms and bedrooms are  correlated (if bedrooms increase  also bathrooms tend to increase)

# Modeling

### - Feature Engineering


    - SelectkBest

In [ ]:
#using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 2)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#using my function for RFE
top_rfe = m.select_rfe(X_train_scaled, y_train, 2,LinearRegression() )

### - Modeling

In [ ]:
#plot our target
sns.displot(y_train,  kind="kde", height =10 ,aspect =1, color = 'red')
plt.xlabel("Value")
plt.ylabel("Number of properties")
plt.show()

In [ ]:
y_train.describe()

In [ ]:
#conver y_train y _ validate to df
y_train = pd.DataFrame( {'actual': y_train})
y_validate = pd.DataFrame( {'actual': y_validate})

**Baseline**

I wil check mean and median for y_train. I will choose median, because as you can se in the plot of the target there are outliers

In [ ]:
y_train.mean(), y_train.median()

In [ ]:
#create baseline using mean (I'm using my function to calculate rmse)
tra_m = eval.baseline_errors(y_train, 'actual', 'mean')
tra_m['rmse']

In [ ]:
#let's calculate baseline in validate
val_m = eval.baseline_errors(y_validate, 'actual', 'mean')
val_m['rmse']

In [ ]:
#create baseline using median
tra = eval.baseline_errors(y_train, 'actual', 'median')
tra['rmse']

In [ ]:
#using median in validate
val = eval.baseline_errors(y_validate, 'actual', 'median')
val['rmse']

**I will select mean for my baseline**

In [ ]:
y_train.head()

In [ ]:
metric_df = pd.DataFrame(data = [{
    'model': 'mean_baseline',
    'rmse_validate': val_m['rmse'],
    'r^2_validate' : val_m['r2']}])

In [ ]:
metric_df

#### Model1: LinearRegression (OLS)

In [ ]:
def create_model (X_df_scaled, y_df, actual, method, name):
    '''
    takes in features scaled df, target df, name of actual target, 
    type of method and the name of the selected method and 
    returns a dictionary that contains calculated regression errors.
    
    X_df_scaled : df that contains scaled featues
    y_df: target df
    actual: name of the column where is actual value of the target
    mehod: type of method to create the model object
    name: enter the new name for your model
    
    Example:
    create_model(X_train_scaled[top_sb], y_train, 'actual', LinearRegression(normalize=True), 'modelOLS' )
    '''
    # fit the thing
    method.fit(X_df_scaled, y_df[actual])

    # predict train
    y_df[name] = method.predict(X_df_scaled)

    #calculate regression errors using a created function
    train_eval = eval.regression_errors(y_df, actual, name)

    return train_eval


In [ ]:
y_train.head()

In [ ]:
top_sb

In [ ]:
#using selected features with selectk best
ols_sb = m.create_model(X_train_scaled[top_sb], y_train, 'actual', LinearRegression(normalize=True), 'modelOLS' )
ols_sb['rmse']

In [ ]:
#using my function for RFE
top_rfe = m.select_rfe(X_train_scaled, y_train, 2,LinearRegression(normalize=True) )


In [ ]:
ols_rfe = m.create_model(X_train_scaled[top_rfe], y_train, 'actual', LinearRegression(normalize=True), 'modelOLS' )
ols_rfe['rmse']

In [ ]:
#I will calcaulate validate with top_rfe 
ols_val = create_model(X_validate_scaled[top_rfe], y_validate, 'actual', LinearRegression(normalize=True), 'modelOLS' )

In [ ]:
print('Train rmse:  ',ols_rfe['rmse'])
print('validate rmse', ols_val['rmse'])

In [ ]:
metric_df = metric_df.append(
    {
    'model': 'ols',
    'rmse_validate': ols_val['rmse'],
    'r^2_validate' : ols_val['r2']}, ignore_index=True)

### LassoLars

In [ ]:
#using selected features with selectk best
lasso_sb = m.create_model(X_train_scaled[top_sb], y_train, 'actual', LassoLars(alpha=1), 'model_lasso' )
lasso_sb['rmse']

In [ ]:
#using my function for RFE
top_rfe = m.select_rfe(X_train_scaled, y_train, 2,  LassoLars(alpha=1) )

In [ ]:
lasso_rfe = m.create_model(X_train_scaled[top_rfe], y_train, 'actual', LassoLars(alpha=1), 'model_lasso' )
lasso_rfe['rmse']

In [ ]:
#validate
lasso_val = m.create_model(X_validate_scaled[top_rfe], y_validate, 'actual', LassoLars(alpha=1), 'model_lasso' )

In [ ]:
print('Train rmse:  ',lasso_rfe['rmse'])
print('validate rmse', lasso_val['rmse'])

In [ ]:
metric_df = metric_df.append(
    {
    'model': 'lasso',
    'rmse_validate': lasso_val['rmse'],
    'r^2_validate' : lasso_val['r2']}, ignore_index = True)


### TweedieRegressor (GLM)


In [ ]:
#train
glm_train = m.create_model(X_train_scaled[top_rfe], y_train, 
                           'actual', TweedieRegressor(power=1, alpha=0), 'model_glm' )

In [ ]:
#validate
glm_val = m.create_model(X_validate_scaled[top_rfe], y_validate, 
                         'actual', TweedieRegressor(power=1, alpha=0), 'model_glm' )



In [ ]:
print('Train rmse:  ',glm_train['rmse'])
print('validate rmse', glm_val['rmse'])

In [ ]:
metric_df = metric_df.append(
    {
    'model': 'glm',
    'rmse_validate': glm_val['rmse'],
    'r^2_validate' : glm_val['r2']}, ignore_index = True)

### Polynomial Regression

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 2) 

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate_scaled)
X_test_degree2 = pf.transform(X_test_scaled)

In [ ]:
#train
pol_reg_train =  m.create_model(X_train_degree2, y_train, 
                                'actual', LinearRegression(normalize=True), 'model_polreg' )

In [ ]:
#validate
pol_reg_val =  m.create_model(X_validate_degree2, 
                              y_validate, 'actual',LinearRegression(normalize=True), 'model_polreg' )

In [ ]:
print('Train rmse:  ',pol_reg_train['rmse'])
print('validate rmse', pol_reg_val['rmse'])

In [ ]:
metric_df = metric_df.append(
    {
    'model': 'pol_reg',
    'rmse_validate': pol_reg_val['rmse'],
    'r^2_validate' : pol_reg_val['r2']},  ignore_index = True)

In [ ]:
metric_df

In [ ]:
# models= {
#     'ols': LinearRegression(normalize=True),
#     'lasso': LassoLars(alpha=1),
#      'tr':   TweedieRegressor(power=1, alpha=0)}
# for k,v in models.items():
#     name = 'model_' + k
#     met = create_model(X_train_scaled[top_sb], y_train, 'actual', v , name )
#     metric_df = metric_df.append(
#     {
#     'model': name,
#     'rmse': met['rmse'],
#     'r^2' : met['r2']}